In [0]:
from pyspark.sql.functions import *

In [0]:
spark.conf.set("fs.azure.account.auth.type.healthexpenditure.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.healthexpenditure.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.healthexpenditure.dfs.core.windows.net","app_id")
spark.conf.set("fs.azure.account.oauth2.client.secret.healthexpenditure.dfs.core.windows.net", "secret_key")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.healthexpenditure.dfs.core.windows.net", "https://login.microsoftonline.com/tenent_id/oauth2/token")

In [0]:
expnd_df =spark.read.format("parquet")\
    .option("header",True)\
        .option("inferSchema",True)\
            .load("abfss://processed-data@healthexpenditure.dfs.core.windows.net/australian_health_expenditure/") \
            .withColumnRenamed("eofy", "end_of_fy") \
            .withColumnRenamed("jurisdiction", "expnd_jurisdiction")

In [0]:
expnd_df.display()

Year,expnd_jurisdiction,Sector,Area_of_Expenditure,Broad_source_of_funds,Source_of_funds,Current_amount,Constant_amount,end_of_fy,ingestion_date
2011-12,ACT,Capital expenditure,Capital expenditure,Australian Government,DoHAC and other,7845557.56,7405237.78,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Capital expenditure,Capital expenditure,Non-government,Other source of funds,25878223.51,29469301.22,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Capital expenditure,Capital expenditure,State and local government,State and local government,174000000.00,195754403.88,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Hospitals,Private hospitals,Australian Government,Department of Veterans' Affairs,28413710.10,34894604.09,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Hospitals,Private hospitals,Australian Government,DoHAC and other,2329108.03,2853160.18,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Hospitals,Private hospitals,Australian Government,Premium rebates,33089182.56,40636506.85,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Hospitals,Private hospitals,Non-government,Health insurance funds,64370521.03,79052817.75,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Hospitals,Private hospitals,Non-government,Individuals,14565797.10,17888115.33,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Hospitals,Private hospitals,Non-government,Other source of funds,8986949.88,11043114.14,Jun-12,2025-01-29T03:20:12.784Z
2011-12,ACT,Hospitals,Private hospitals,State and local government,State and local government,4734000.00,5813779.87,Jun-12,2025-01-29T03:20:12.784Z


In [0]:
fund_source_df = expnd_df \
    .groupBy("year", "expnd_jurisdiction", "source_of_funds", "end_of_fy") \
    .agg(sum("current_amount").alias("total_amount"))

In [0]:
pop_df =spark.read.format("parquet")\
    .option("header",True)\
        .option("inferSchema",True)\
            .load("abfss://processed-data@healthexpenditure.dfs.core.windows.net/austrialian_population_by_state/") \
.withColumnRenamed("jurisdiction", "pop_jurisdiction")

In [0]:
join_condition = [ fund_source_df.end_of_fy == pop_df.time, fund_source_df.expnd_jurisdiction == pop_df.pop_jurisdiction ]
fund_source_with_pop_df = fund_source_df.join(pop_df, join_condition)

In [0]:

final_df = fund_source_with_pop_df \
                .withColumn("amount_per_person", expr("total_amount / population")) \
                .withColumn("created_date", current_timestamp())

final_df.display()

year,expnd_jurisdiction,source_of_funds,end_of_fy,total_amount,time,pop_jurisdiction,Population,ingestion_date,amount_per_person,created_date
2012-13,NT,Health insurance funds,Jun-13,56485781.49,Jun-13,NT,512231,2025-01-28T23:53:58.724Z,110.2740394275,2025-01-29T18:40:16.245Z
2015-16,NSW,State and local government,Jun-16,12322534590.08,Jun-16,NSW,7732858,2025-01-28T23:53:58.724Z,1593.5291440862,2025-01-29T18:40:16.245Z
2016-17,NT,Individuals,Jun-17,241272962.90,Jun-17,NT,526762,2025-01-28T23:53:58.724Z,458.0303114120,2025-01-29T18:40:16.245Z
2016-17,TAS,Health insurance funds,Jun-17,352034990.01,Jun-17,TAS,2585720,2025-01-28T23:53:58.724Z,136.1458278584,2025-01-29T18:40:16.245Z
2016-17,VIC,Department of Veterans' Affairs,Jun-17,550197168.39,Jun-17,VIC,6302608,2025-01-28T23:53:58.724Z,87.2967457900,2025-01-29T18:40:16.245Z
2012-13,NT,State and local government,Jun-13,1092118895.46,Jun-13,NT,512231,2025-01-28T23:53:58.724Z,2132.0827819089,2025-01-29T18:40:16.245Z
2013-14,NSW,Department of Veterans' Affairs,Jun-14,1151172709.90,Jun-14,NSW,7508353,2025-01-28T23:53:58.724Z,153.3189382412,2025-01-29T18:40:16.245Z
2021-22,NT,Other source of funds,Jun-22,131495893.27,Jun-22,NT,571007,2025-01-28T23:53:58.724Z,230.2877079791,2025-01-29T18:40:16.245Z
2021-22,QLD,Individuals,Jun-22,6048126679.51,Jun-22,QLD,5320711,2025-01-28T23:53:58.724Z,1136.7139992212,2025-01-29T18:40:16.245Z
2016-17,TAS,Individuals,Jun-17,663444152.73,Jun-17,TAS,2585720,2025-01-28T23:53:58.724Z,256.5800445253,2025-01-29T18:40:16.245Z


In [0]:
final_df.write.format("parquet")\
    .mode("overwrite")\
    .option("path","abfss://presentation-data@healthexpenditure.dfs.core.windows.net/fund_source_with_pop")\
    .save()